### Part 2

Estimate the emission parameters from the training set using MLE

In [2]:
def emission_params(tokens_list: list, y: str, x: str, special_token='#UNK#'):
    y_count = 0
    x_given_y_count = 0
    x_count = 0
    for token in tokens_list: 
        if token[1] == y:
            y_count += 1
            if token[0] == x:
                x_given_y_count += 1
        if token[0] == x:
            x_count += 1
    
    if x_count == 0:
        return emission_params(tokens_list, y, special_token)
    return float(x_given_y_count) / y_count

In [3]:
# Example ussage
import os

data = [['a', 'O'], ['b', 'O'], ['a', 'I'], ['c', 'O']]
emission_params(tokens_list=data, y='O', x='a')

# with open('./EN/train') as train_file:
#     read_data = train_file.read()
#     read_data = os.linesep.join([s for s in read_data.splitlines() if s])
#     data = list(map(lambda x: x.split(' '),read_data.split('\n')))
# emission_params(tokens_list=data, y='O', x='a')

0.3333333333333333

Modify training set to replace words that appear less than k times with special token. Apply this to the emission parameters prediction function with k = 3

In [4]:
def clean_data(tokens_list, k = 1, special_token = '#UNK#'):
    token_freq = {}
    for token in tokens_list:
        if token[0] not in token_freq: 
            token_freq[token[0]] = 1
        else:
            token_freq[token[0]] += 1
    for i in range(len(tokens_list)):
        if token_freq[tokens_list[i][0]] < k:
            tokens_list[i][0] = special_token
    
    return tokens_list

def emission_params_clean_data(tokens_list: list, y: str, x: str):
    tokens_list = clean_data(tokens_list = tokens_list, k = 3)
    return emission_params(tokens_list, y, x)

In [5]:
# Example usage
data = [['a', 'O'], ['b', 'O'], ['a', 'I'], ['c', 'O'], ['a', 'O']]
emission_params_clean_data(tokens_list=data, y='O', x='a')

0.5

Sentiment analysis system that produces the tag for input

In [10]:
def predict_tag(tokens_list: list, x: str):
    y_tags = ['O', 'B-positive', 'I-positive', 'B-neutral', 'I-neutral', 'B-negative', 'I-negative']
    score = 0.0
    y_tag = None
    for y in y_tags:
        y_score = emission_params_clean_data(tokens_list, y, x)
        if (y_score > score):
            y_tag = y
            score = y_score 
    return y_tag

# clean data before predicting the tag
def predict_tag_optimized(tokens_list: list, x: str):
    y_tags = ['O', 'B-positive', 'I-positive', 'B-neutral', 'I-neutral', 'B-negative', 'I-negative']
    score = 0.0
    y_tag = None
    for y in y_tags:
        y_score = emission_params(tokens_list, y, x)
        if (y_score > score):
            y_tag = y
            score = y_score 
    return y_tag
languages = ['EN', 'SG', 'CN', 'FR']

for l in languages:
    with open("./{}/train".format(l)) as train_file:
        read_data = train_file.read()
        read_data = os.linesep.join([s for s in read_data.splitlines() if s])
        data = list(map(lambda x: x.split(' '),read_data.split('\n')))
        data_cleaned = clean_data(tokens_list = data, k = 3)

    with open("./{}/dev.in".format(l)) as in_file, open("./{}/dev.p2.out".format(l), 'w+') as out_file:
        for line in in_file:
            word = line.strip()
            if (word == ''):
                out_file.write("\n")
            else:
                out_file.write("{} {}\n".format(word, predict_tag_optimized(data_cleaned, word)))
    print("Finished: {}".format(l))

Finished: EN
Finished: SG
Finished: CN
Finished: FR


In [12]:
for l in languages:
    output = os.popen("python EvalScript/evalResult.py {0}/dev.out {0}/dev.p2.out".format(l)).read()
    print("Language: {}".format(l))
    print(output)
    print("----------------------")

Language: EN

#Entity in gold data: 226
#Entity in prediction: 1201

#Correct Entity : 165
Entity  precision: 0.1374
Entity  recall: 0.7301
Entity  F: 0.2313

#Correct Sentiment : 71
Sentiment  precision: 0.0591
Sentiment  recall: 0.3142
Sentiment  F: 0.0995

----------------------
Language: SG

#Entity in gold data: 1382
#Entity in prediction: 6599

#Correct Entity : 794
Entity  precision: 0.1203
Entity  recall: 0.5745
Entity  F: 0.1990

#Correct Sentiment : 315
Sentiment  precision: 0.0477
Sentiment  recall: 0.2279
Sentiment  F: 0.0789

----------------------
Language: CN

#Entity in gold data: 362
#Entity in prediction: 3318

#Correct Entity : 183
Entity  precision: 0.0552
Entity  recall: 0.5055
Entity  F: 0.0995

#Correct Sentiment : 57
Sentiment  precision: 0.0172
Sentiment  recall: 0.1575
Sentiment  F: 0.0310

----------------------
Language: FR

#Entity in gold data: 223
#Entity in prediction: 1149

#Correct Entity : 182
Entity  precision: 0.1584
Entity  recall: 0.8161
Entity  F